# An Introduction to Numerical Computing with Python

While the Python language is an excellent tool for general-purpose programming, with a highly readable syntax, rich and powerful data types (strings, lists, sets, dictionaries, arbitrary length integers, etc) and a very comprehensive standard library, it was not designed specifically for mathematical and scientific computing.  Neither the language nor its standard library have facilities for the efficient representation of multidimensional datasets, tools for linear algebra and general matrix manipulations (an essential building block of virtually all technical computing), nor any data visualization facilities.

In particular, Python lists are very flexible containers that can be nested arbitrarily deep and which can hold any Python object in them, but they are poorly suited to represent efficiently common mathematical constructs like vectors and matrices.  In contrast, much of our modern heritage of scientific computing has been built on top of libraries written in the Fortran language, which has native support for vectors and matrices as well as a library of mathematical functions that can efficiently operate on entire arrays at once.

## Basics of Numpy arrays

We now turn our attention to the Numpy library, which forms the base layer for the entire 'scipy ecosystem'.  Once you have installed numpy, you can import it as

In [ ]:
import numpy as np

In [ ]:
x = range(50000)
y = np.arange(50000)

%timeit [e**2  for e in x]
%timeit y**2

### The Numpy array structure
<center>
<img src="files/array_memory_strides.png" width=70%>
</center>

### Arrays vs lists

As mentioned above, the main object provided by numpy is a powerful array.  We'll start by exploring how the numpy array differs from Python lists.  We start by creating a simple list and an array with the same contents of the list:

In [ ]:
lst = [10, 20, 30, 40]
arr = np.array([10, 20, 30, 40])

Elements of a one-dimensional array are accessed with the same syntax as a list:

In [ ]:
lst[0]

In [ ]:
arr[0]

In [ ]:
arr[-1]

In [ ]:
arr[2:]

The first difference to note between lists and arrays is that arrays are *homogeneous*; i.e. all elements of an array must be of the same type.  In contrast, lists can contain elements of arbitrary type. For example, we can change the last element in our list above to be a string:

In [ ]:
lst[-1] = 'a string inside a list'
lst

but the same can not be done with an array, as we get an error message:

In [ ]:
arr[-1] = 'a string inside an array'

### Array memory representation

The information about the type of an array is contained in its *dtype* attribute:

In [ ]:
x = np.array([[1, 2], [3, 4]], dtype=np.uint8)
print(x)

In [ ]:
[b for b in bytes(x.data)]

In [ ]:
arr = np.array([10, 20, 123123])
arr.dtype

In [ ]:
[b for b in bytes(arr.data)]

Once an array has been created, its dtype is fixed and it can only store elements of the same type.  For this example where the dtype is integer, if we store a floating point number it will be automatically converted into an integer:

In [ ]:
arr[-1] = 1.234
arr

Strange things can also happen when manipulating values in an array:

In [ ]:
x = np.array([0, 127, 255], dtype=np.uint8)
print(x)

In [ ]:
x + 1

In [ ]:
x - 1

### Array creation

Above we created an array from an existing list; now let us now see other ways in which we can create arrays, which we'll illustrate next.  A common need is to have an array initialized with a constant value, and very often this value is 0 or 1 (suitable as starting value for additive and multiplicative loops respectively); `zeros` creates arrays of all zeros, with any desired dtype:

In [ ]:
np.zeros((5, 5), dtype=np.float64)

In [ ]:
np.zeros((2, 3), dtype=np.int64)

In [ ]:
np.zeros(3, dtype=complex)

and similarly for `ones`:

In [ ]:
np.ones(5)

Then there are the `linspace` and `logspace` functions to create linearly and logarithmically-spaced grids, respectively, with a fixed number of points and including both ends of the specified interval:

In [ ]:
np.linspace(0, 1, 5)   # start, stop, num

In [ ]:
np.logspace(1, 4, 4)  # Logarithmic grid between 10^1 and 10^4

Finally, it is often useful to create arrays with random numbers that follow a specific distribution.  The `np.random` module provides several random number generators.  For example, here we produce an array of 5 random samples taken from a standard normal distribution (0 mean and variance 1):

In [ ]:
rng = np.random.RandomState(0)  # <-- seed value, do not have to specify, but useful for reproducibility

In [ ]:
rng.normal(loc=5, scale=1, size=5)

Or the same, but from a uniform distribution:

In [ ]:
uni = rng.uniform(-10, 10, size=5)  # 5 random numbers, picked from a uniform distribution between -10 and 10
print(uni)

## Indexing with other arrays

Above we saw how to index arrays with single numbers and slices, just like Python lists.  But arrays allow for a more sophisticated kind of indexing which is very powerful: you can index an array with another array, and in particular with an array of boolean values.  This is particluarly useful to extract information from an array that matches a certain condition.

Consider for example that in the array `uni` we want to replace all values above 0 with the value 10.  We can do so by first finding the *mask* that indicates where this condition is true or false:

In [ ]:
mask = uni > 0
mask

Now that we have this mask, we can use it to either read those values or to reset them to 0:

In [ ]:
print('Array:', uni)
print('Masked array:', uni[mask])

In [ ]:
uni[mask] = 10
print(uni)

### Arrays with more than one dimension

Most of the array creation methods can be used to construct >1D arrays:

In [ ]:
np.zeros((3, 4))

We can also reshape arrays to fit the desired shape:

In [ ]:
np.arange(12)

In [ ]:
arr = np.arange(12).reshape((3, 4))
arr

With two-dimensional arrays we start seeing the power of numpy: while a nested list can be indexed using repeatedly the `[ ]` operator, multidimensional arrays support a more direct indexing syntax with a single `[ ]` and a set of indices separated by commas:

In [ ]:
arr[0, 1]

In [ ]:
arr[:, 0]

If you only provide one index, then you will get an array with one less dimension containing that row:

In [ ]:
print('First row:  ', arr[0])
print('Second row: ', arr[1])

### Other array properties

Now that we have seen how to create arrays with more than one dimension, let's take a look at some other properties.

In [ ]:
print('Data type                :', arr.dtype)
print('Total number of elements :', arr.size)
print('Number of dimensions     :', arr.ndim)
print('Shape (dimensionality)   :', arr.shape)
print('Memory used (in bytes)   :', arr.nbytes)

Arrays also have many useful methods, some especially useful ones are:

In [ ]:
print('Minimum and maximum             :', arr.min(), arr.max())
print('Sum and product of all elements :', arr.sum(), arr.prod())
print('Mean and standard deviation     :', arr.mean(), arr.std())

For these methods, the above operations area all computed on all the elements of the array.  But for a multidimensional array, it's possible to do the computation along a single dimension, by passing the `axis` parameter; for example:

In [ ]:
print('For the following array:\n', arr)
print('The sum of elements along the rows is    :', arr.sum(axis=1))
print('The sum of elements along the columns is :', arr.sum(axis=0))

As you can see in this example, the value of the `axis` parameter is the dimension which will be *consumed* once the operation has been carried out.  This is why to sum along the rows we use `axis=0`.

Another widely used property of arrays is the `.T` attribute, which allows you to access the transpose of the array (NumPy does this without making a copy of the array, by manipulating its strides):

In [ ]:
print('Array:\n', arr)
print('Transpose:\n', arr.T)

We don't have time here to look at all the methods and properties of arrays, here's a complete list.  Simply try exploring some of these IPython to learn more, or read their description in their docstrings or the [Numpy documentation](http://docs.scipy.org/doc/numpy/reference/):

    arr.T             arr.copy          arr.getfield      arr.put           arr.squeeze
    arr.all           arr.ctypes        arr.imag          arr.ravel         arr.std
    arr.any           arr.cumprod       arr.item          arr.real          arr.strides
    arr.argmax        arr.cumsum        arr.itemset       arr.repeat        arr.sum
    arr.argmin        arr.data          arr.itemsize      arr.reshape       arr.swapaxes
    arr.argsort       arr.diagonal      arr.max           arr.resize        arr.take
    arr.astype        arr.dot           arr.mean          arr.round         arr.tofile
    arr.base          arr.dtype         arr.min           arr.searchsorted  arr.tolist
    arr.byteswap      arr.dump          arr.nbytes        arr.setasflat     arr.tostring
    arr.choose        arr.dumps         arr.ndim          arr.setfield      arr.trace
    arr.clip          arr.fill          arr.newbyteorder  arr.setflags      arr.transpose
    arr.compress      arr.flags         arr.nonzero       arr.shape         arr.var
    arr.conj          arr.flat          arr.prod          arr.size          arr.view
    arr.conjugate     arr.flatten       arr.ptp           arr.sort          

## Operating with arrays

Standard mathematical operations Just Work (TM):

In [ ]:
arr1 = np.arange(4)
arr2 = np.arange(10, 14)
print(arr1, '+', arr2, '=', arr1 + arr2)

Note, that, unlike in MATLAB, operations are performed element-wise:

In [ ]:
print(arr1, '*', arr2, '=', arr1 * arr2)

While this means that, in principle, arrays must always match in their dimensionality in order for an operation to be valid, numpy will *broadcast* dimensions when possible.  For example, suppose that you want to add the number 1.5 to `arr1`, this works:

In [ ]:
arr1 + 3

<img src="files/broadcast_1D.png"/>

### The broadcasting rules

This broadcasting behavior is powerful, especially because when numpy broadcasts to create new dimensions or to 'stretch' existing ones, it doesn't replicate the data.  In the example above the operation is carried *as if* the 3 was a 1-d array with 3 in all of its entries, but no actual array was ever created.  This can save memory in cases when the arrays in question are large, with significant performance implications.

The general rule is: when operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing dimensions, and works its way forward, creating dimensions of length 1 as needed. Two dimensions are considered compatible when

* they are equal or either is None or one
* either dimension is 1 or ``None``, or if dimensions are equal

If these conditions are not met, a `ValueError: frames are not aligned` exception is thrown, indicating that the arrays have incompatible shapes. The size of the resulting array is the maximum size along each dimension of the input arrays.

Examples below:

```
(9, 5)   (9, 5)   (9, 5)   (9, 1)
   ( )   (9, 1)   (   5)   (   5)
------   ------   ------   ------
(9, 5)   (9, 5)   (9, 5)   (9, 5)

```

<img src="files/broadcast_rougier.png"/>

Sketch from [Nicolas Rougier's NumPy tutorial](http://www.labri.fr/perso/nrougier/teaching/numpy/numpy.html)

### Visual illustration of broadcasting
<center>
<img src="files/numpy_broadcasting.svg" width=80%>
</center>

Sometimes, it is necessary to modify arrays before they can be used together.  Numpy allows you to add new axes to an array by indexing with `np.newaxis`:

In [ ]:
c = np.arange(5)
d = np.arange(6)

print(c.shape)
print(d.shape)

c + d

In [ ]:
c = np.arange(5)
d = np.arange(6)

c = c[:, np.newaxis]

print(c.shape)
print('  ', d.shape)
print('-------')
print((c + d).shape)
 
#   d d d d d d
#
# c              c c c c c c   d d d d d d
# c              c c c c c c   d d d d d d
# c     +      = c c c c c c + d d d d d d
# c              c c c c c c   d d d d d d
# c              c c c c c c   d d d d d d

c + d

For the full broadcasting rules, please see the official Numpy docs, which describe them in detail and with more complex examples.

Also see: [G-Node Summer School Advanced NumPy tutorial](https://github.com/stefanv/teaching/blob/master/2014_assp_split_numpy/numpy_advanced.ipynb)

As we mentioned before, Numpy ships with a full complement of mathematical functions that work on entire arrays, including logarithms, exponentials, trigonometric and hyperbolic trigonometric functions, etc.  Furthermore, scipy ships a rich special function library in the `scipy.special` module that includes Bessel, Airy, Fresnel, Laguerre and other classical special functions.  For example, sampling the sine function at 100 points between $0$ and $2\pi$ is as simple as:

In [ ]:
x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)

## Linear algebra in numpy

Numpy ships with a basic linear algebra library, and all arrays have a `dot` method whose behavior is that of the scalar dot product when its arguments are vectors (one-dimensional arrays) and the traditional matrix multiplication when one or both of its arguments are two-dimensional arrays:

In [ ]:
v1 = np.array([2, 3, 4])
v2 = np.array([1, 0, 1])
print(v1, '·', v2, '=', v1.dot(v2))

In [ ]:
A = np.arange(6).reshape(2, 3)
print(A, 'x', v1, '=', A.dot(v1))

For matrix-matrix multiplication, the same dimension-matching rules must be satisfied, e.g. consider the difference between $A \times A^T$:

In [ ]:
print(A.dot(A.T))

and $A^T \times A$:

In [ ]:
print(A.T.dot(A))

Furthermore, the `numpy.linalg` module includes additional functionality such as determinants, matrix norms, Cholesky, eigenvalue and singular value decompositions, etc.  For even more linear algebra tools, `scipy.linalg` contains the majority of the tools in the classic LAPACK libraries as well as functions to operate on sparse matrices.  We refer the reader to the Numpy and Scipy documentations for additional details on these.

## Reading and writing arrays to disk

Numpy lets you read and write arrays into files in a number of ways.  In order to use these tools well, it is critical to understand the difference between a *text* and a *binary* file containing numerical data.  In a text file, the number $\pi$ could be written as "3.141592653589793", for example: a string of digits that a human can read, with in this case 15 decimal digits.  In contrast, that same number written to a binary file would be encoded as 8 characters (bytes) that are not readable by a human but which contain the exact same data that the variable `pi` had in the computer's memory.  

The tradeoffs between the two modes are thus:

* Text mode: occupies more space, precision can be lost (if not all digits are written to disk), but is readable and editable by hand with a text editor.  Can *only* be used for one- and two-dimensional arrays.

* Binary mode: compact and exact representation of the data in memory, can't be read or edited by hand.  Arrays of any size and dimensionality can be saved and read without loss of information.

First, let's see how to read and write arrays in text mode.  The `np.savetxt` function saves an array to a text file, with options to control the precision, separators and even adding a header:

In [ ]:
arr = np.arange(10).reshape(2, 5)
print(arr)                            
np.savetxt('test.out', arr)

In [ ]:
!cat test.out

And this same type of file can then be read with the matching `np.loadtxt` function:

In [ ]:
arr2 = np.loadtxt('test.out')
print(arr2)

For binary data, Numpy provides the `np.save` and `np.savez` routines.  The first saves a single array to a file with `.npy` extension, while the latter can be used to save a *group* of arrays into a single file with `.npz` extension.  The files created with these routines can then be read with the `np.load` function.

Let us first see how to use the simpler `np.save` function to save a single array:

In [ ]:
np.save('test.npy', arr)
# Now we read this back
arr_loaded = np.load('test.npy')

print(arr)
print(arr_loaded)

print(arr_loaded.dtype)

# Let's see if any element is non-zero in the difference.
# A value of True would be a problem.
print('Any differences?', np.any(arr - arr_loaded))

Now let us see how the `np.savez_compressed` function works.

In [ ]:
np.savez_compressed('test.npz', first=arr, second=arr2)
arrays = np.load('test.npz')
arrays.files

The object returned by `np.load` from an `.npz` file works like a dictionary:

In [ ]:
arrays['first']

This `.npz` format is a very convenient way to package compactly and without loss of information, into a single file, a group of related arrays that pertain to a specific problem.  At some point, however, the complexity of your dataset may be such that the optimal approach is to use one of the standard formats in scientific data processing that have been designed to handle complex datasets, such as NetCDF or HDF5.  

Fortunately, there are tools for manipulating these formats in Python, and for storing data in other ways such as databases.  A complete discussion of the possibilities is beyond the scope of this tutorial, but of particular interest for scientific users we at least mention the following:

* The `scipy.io` module contains routines to read and write Matlab files in `.mat` format and files in the NetCDF format that is widely used in certain scientific disciplines.

* For manipulating files in the HDF5 format, there are two excellent options in Python: the **PyTables** project offers a high-level, object oriented approach to manipulating HDF5 datasets, while the **h5py** project offers a more direct mapping to the standard HDF5 library interface.  Both are excellent tools; if you need to work with HDF5 datasets you should read some of their documentation and examples and decide which approach is a better match for your needs.